In [36]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
from random import randint
from scipy.ndimage.filters import laplace 
import scipy.stats as st
from scipy.signal import correlate2d
import collections
%matplotlib inline
%load_ext snakeviz

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz


In [3]:
def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

def P2R(radii, angles):
    return radii * np.exp(1j*angles)

def R2P(x):
    return np.abs(x), np.angle(x)

def LaplacianInterriorDomain(Image):
    M=len(Image)
    N=len(Image[0])
    u=np.copy(Image)
    DeltaU=laplace(u)
    v1=np.zeros([M,N])
    v2=np.zeros([M,N])
    for y in range(N):
        v1[0,y]=u[M-1,y]-u[0,y]
        v1[M-1,y]=u[0,y]-u[M-1,y]
    for x in range(M):
        v2[x,0]=u[x,N-1]-u[x,0]
        v2[x,N-1]=u[x,0]-u[x,N-1]
    v=v1+v2
    DeltaU_V=DeltaU-v
    return(DeltaU_V)

In [23]:
def AddNoise(Image,Noise):
    ImageFFT=(np.fft.fft2(Image))
    norm, phase = R2P(ImageFFT)
    NImageFFT=P2R(norm,phase+Noise)
    return((np.abs(np.fft.ifft2(NImageFFT))))

def GenerateNoise(Image):
    RandomPhase=np.random.rand(len(Image),len(Image[0]))*2*math.pi-np.ones([len(Image),len(Image[0])])*math.pi
    return RandomPhase

def RPN(Image):
    Noise=GenerateNoise(Image)
    NewImage=np.zeros(Image.shape)
    NewImage[:,:,0]=AddNoise(Image[:,:,0],Noise)
    NewImage[:,:,1]=AddNoise(Image[:,:,1],Noise)
    NewImage[:,:,2]=AddNoise(Image[:,:,2],Noise)
    return(NewImage)

def PeriodicComponent(Image):
    M=len(Image)
    N=len(Image[0])
    Lap=LaplacianInterriorDomain(Image)
    LapFFT=np.fft.fft2(Lap)
    PFFT=np.zeros([M,N],dtype=np.complex)
    for i in range(M):
        for j in range(N):
            if (i==0 and j==0):
                PFFT[i,j]=np.sum(np.ravel(Image))
            else:
                PFFT[i,j]=LapFFT[i,j]/(2*np.cos(2*i*math.pi/M)+2*np.cos(2*j*math.pi/N)-4)
    P=np.fft.ifft2(PFFT)
    return(P)

def ExpandTexture(Image,S0,S1):
    M=len(Image)
    N=len(Image[0])
    mR=np.mean(Image[:,:,0].ravel())
    mG=np.mean(Image[:,:,1].ravel())
    mB=np.mean(Image[:,:,2].ravel())
    PR=np.abs(PeriodicComponent(Image[:,:,0]))
    PG=np.abs(PeriodicComponent(Image[:,:,1]))
    PB=np.abs(PeriodicComponent(Image[:,:,2]))
    Texture=np.ones([S0,S1,3])
    Texture[:,:,0]=Texture[:,:,0]*mR
    Texture[:,:,1]=Texture[:,:,1]*mG
    Texture[:,:,2]=Texture[:,:,2]*mB
    Texture[(S0/2-M/2):(S0/2-M/2+M),(S1/2-N/2):(S1/2-N/2+N),0]=(PR-mR*np.ones([M,N]))*np.sqrt(S0*S1/(M*N))+mR*np.ones([M,N])
    Texture[(S0/2-M/2):(S0/2-M/2+M),(S1/2-N/2):(S1/2-N/2+N),1]=(PG-mG*np.ones([M,N]))*np.sqrt(S0*S1/(M*N))+mG*np.ones([M,N])
    Texture[(S0/2-M/2):(S0/2-M/2+M),(S1/2-N/2):(S1/2-N/2+N),2]=(PB-mB*np.ones([M,N]))*np.sqrt(S0*S1/(M*N))+mB*np.ones([M,N])
    R=RPN(Texture)
    R[R<0]=0
    R[R>1]=0.99999
    return R

In [37]:
%%snakeviz
img=np.array(mpimg.imread('../../net.png')[:,:,:3])
# plt.imshow(img,cmap='gray')
# plt.axis('off')
# plt.show()
b=ExpandTexture(np.array(img,dtype=np.float),300,300)
# plt.imshow(b,cmap='gray')
# plt.axis('off')
# plt.show()

 
*** Profile stats marshalled to file u'c:\\users\\jqxs\\appdata\\local\\temp\\tmpghordu'. 
